# Example Seldon Core Deployments using Ksonnet


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [ksonnet client](https://ksonnet.io/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

## Setup

When you have a running minikube cluster run:


In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

## Install Seldon Core
Create a ksonnet app and install the prototypes from our registry.

In [ ]:
!ks init my-ml-deployment 

In [ ]:
!cd my-ml-deployment && \
    ks registry add seldon-core ../../seldon-core && \
    ks pkg install seldon-core/seldon-core@master && \
    ks generate seldon-core seldon-core --withApife=false --withAmbassador=true --namespace=seldon --withRbac=true

In [ ]:
!cd my-ml-deployment && \
      ks apply default

In [ ]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-ambassador

## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

Install gRPC modules for the prediction protos.

In [ ]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

Illustration of both REST and gRPC requests. 

## Serve Single Model

In [ ]:
!cd my-ml-deployment && \
    ks generate seldon-serve-simple-v1alpha2 mymodel --image seldonio/mock_classifier:1.0 && \
    ks apply default -c mymodel

In [ ]:
!cd my-ml-deployment && \
    ks show default -c mymodel

In [ ]:
!kubectl rollout status deploy/mymodel-mymodel-mymodel-0
!kubectl rollout status deploy/mymodel-mymodel-svc-orch

Check status of deployment before continuing. **ReplicasAvailable must be equal to 1**  First time might take some time to download images.

In [ ]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' -n seldon

### Get predictions

In [ ]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [ ]:
rest_request_ambassador("mymodel",API_AMBASSADOR)

#### gRPC Request

In [ ]:
 grpc_request_ambassador("mymodel",API_AMBASSADOR)

## Adding Authentication
We will add the example authentication from the Ambassador tutorial.

In [ ]:
!kubectl apply -f resources/ambassador-auth-service-setup.yaml -n seldon

** Need to wait until running before adding Ambassador config **

In [ ]:
!kubectl get pods -n seldon

In [ ]:
!kubectl apply -f resources/ambassador-auth-service-config.yaml -n seldon

Show failed request when auth is running

In [ ]:
rest_request_ambassador("mymodel",API_AMBASSADOR)

Show successful request with auth

In [ ]:
rest_request_ambassador_auth("mymodel","username","password",API_AMBASSADOR)

In [ ]:
!kubectl delete -f resources/ambassador-auth-service-setup.yaml -n seldon

In [ ]:
!cd my-ml-deployment && \
    ks delete default -c mymodel && \
    ks component rm mymodel

## Serve AB Test

In [ ]:
!cd my-ml-deployment && \
    ks generate seldon-abtest-v1alpha2 myabtest --imageA seldonio/mock_classifier:1.0 --imageB seldonio/mock_classifier:1.0 && \
    ks apply default -c myabtest

In [ ]:
!cd my-ml-deployment && \
    ks show default -c myabtest

In [ ]:
!kubectl rollout status deploy/myabtest-myabtest-svc-orch

Check status of deployment before continuing. **ReplicasAvailable must be equal to 1**  First time might take some time to download images.

In [ ]:
!kubectl get seldondeployments myabtest -o jsonpath='{.status}' -n seldon

### Get predictions

In [ ]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [ ]:
rest_request_ambassador("myabtest",API_AMBASSADOR)

#### gRPC Request

In [ ]:
 grpc_request_ambassador("myabtest",API_AMBASSADOR)

In [ ]:
!cd my-ml-deployment && \
    ks delete default -c myabtest && \
    ks component rm myabtest

## Serve Multi-Armed Bandit

In [ ]:
!cd my-ml-deployment && \
    ks generate seldon-mab-v1alpha2 mymab --imageA seldonio/mock_classifier:1.0 --imageB seldonio/mock_classifier:1.0 && \
    ks apply default -c mymab

In [ ]:
!cd my-ml-deployment && \
    ks show default -c mymab

In [ ]:
!kubectl rollout status deploy/mymab-mymab-svc-orch

Check status of deployment before continuing. **ReplicasAvailable must be equal to 1**  First time might take some time to download images.

In [ ]:
!kubectl get seldondeployments mymab -o jsonpath='{.status}' -n seldon

### Get predictions

In [ ]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [ ]:
rest_request_ambassador("mymab",API_AMBASSADOR)

#### gRPC Request

In [ ]:
 grpc_request_ambassador("mymab",API_AMBASSADOR)

In [ ]:
!cd my-ml-deployment && \
    ks delete default -c mymab && \
    ks component rm mymab

## Serve Model with Outlier Detector

In [ ]:
!cd my-ml-deployment && \
    ks generate seldon-outlier-detector-v1alpha2 myout --image seldonio/mock_classifier:1.0 && \
    ks apply default -c myout

In [ ]:
!cd my-ml-deployment && \
    ks show default -c myout

In [ ]:
!kubectl rollout status deploy/myout-myout-svc-orch
!kubectl rollout status deploy/myout-myout-outlier-detector-1

Check status of deployment before continuing. **ReplicasAvailable must be equal to 1**  First time might take some time to download images.

In [ ]:
!kubectl get seldondeployments myout -o jsonpath='{.status}' -n seldon

### Get predictions

In [ ]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [ ]:
rest_request_ambassador("myout",API_AMBASSADOR)

#### gRPC Request

In [ ]:
 grpc_request_ambassador("myout",API_AMBASSADOR)

In [ ]:
!cd my-ml-deployment && \
    ks delete default -c myout && \
    ks component rm myout

# Tear down

In [ ]:
!cd my-ml-deployment && ks delete default

In [ ]:
!rm -rf my-ml-deployment